In [2]:
import pandas as pd
import numpy as np
import os
import imp
import telephone_analysis
import srilm
import roark
import glob
import shutil
import scipy.stats
imp.reload(telephone_analysis)
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import kenlm

In [3]:
all_runs  = pd.read_csv('output/all_runs.csv')
# run these through each of the language models and merge the results
# make a word level table
# diff the words to see if they are not present in the next generation

In [4]:
all_runs.loc[all_runs.user != "0"].shape[0]

2999

In [5]:
len(np.unique(all_runs.loc[all_runs.user != "0"].user))

266

In [6]:
all_runs.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'chain', 'character_levdau', 'check_time',
       'condition', 'flag_type', 'gold_candidate_transcription',
       'gold_comparison_transcription', 'gold_dist',
       ...
       'initial_gpt2_normal_probability_rank',
       'initial_gpt2_normal_probability_quartile',
       'initial_gpt2_medium_probability',
       'initial_gpt2_medium_probability_rank',
       'initial_gpt2_medium_probability_quartile', 'initial_bert_probability',
       'initial_bert_probability_rank', 'initial_bert_probability_quartile',
       'thread_id', 'chain_length'],
      dtype='object', length=101)

# Word-Level Language Modeling Results

In [7]:
# index in the LM corresponds to the response: user_candidate_transcription. 3192: last input

In [8]:
lm = {}

In [9]:
bnc_knn_lm = srilm.LM("LMs/BNC_merged.LM", lower=True)

In [10]:
lm['bnc_unigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram=True) for x in all_runs['user_candidate_transcription']]

In [11]:
assert(len(lm['bnc_unigram']) == all_runs.shape[0])

In [12]:
lm['bnc_trigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram = False) for x in all_runs['user_candidate_transcription']]

In [13]:
assert(len(lm['bnc_trigram']) == all_runs.shape[0])

In [14]:
lm['roark_scores'] = roark.parse([str(i) for i in all_runs['user_candidate_transcription']], 
    '/home/stephan/utils/incremental-top-down-parser',
                          numWorkers=24, mode='single_word')

Input contains 3193 sentences
Input contains 1982 unique sentences


PermissionError: [Errno 13] Permission denied: '/home/stephan/utils/incremental-top-down-parser/temp/0.input'

In [ ]:
#roark_scores.columns = ['WSJ_Roark_'+x.replace(' ','.') for x in roark_scores.columns]
assert len(lm['roark_scores']) == all_runs.shape[0]

In [ ]:
lm['big_lm_scores'] = telephone_analysis.getBigLMscores(all_runs['user_candidate_transcription'], 'big_lm',
    'big_lm_cache', colname='BigLM_probability', lm_1b_dir='/home/stephan/python/lm_1b', mode='single_word')


In [ ]:
assert(len(lm['big_lm_scores']) == all_runs.shape[0])

In [ ]:
m = kenlm.Model('LMs/deepspeech_5gram.binary')
lm['kenlm_scores'] = all_runs['kenlm_probability'] = [telephone_analysis.getKenLMProb(x, m, mode='single_word') for x in all_runs['user_candidate_transcription']]

In [ ]:
assert(len(lm['kenlm_scores']) == all_runs.shape[0])

In [ ]:
# sanity check: all sentences in all models must have the same number of tokens in the dataframe
languageModelNames = ['bnc_unigram', 'bnc_trigram', 'roark_scores', 'big_lm_scores', 'kenlm_scores']
for i in range(all_runs.shape[0]):
    sentences = [lm[x][i] for x in languageModelNames]
    try:
        numWords = [x.shape[0] for x in sentences]
    except:
        print('Problem counting words')
        import pdb
        pdb.set_trace()
    if not np.allclose(numWords[1:len(numWords)], numWords[0]):
        print('Different number of words')
        import pdb
        pdb.set_trace()

In [ ]:
sentence_store = []
for i in range(all_runs.shape[0]):
    # need to rename what is coming out of each language model
    word_store = [] 
    for languageModelName in languageModelNames:
        df = lm[languageModelName][i].copy()
        try:
            df.columns = [languageModelName+'_'+x for x in df.columns]        
        except:
            import pdb
            pdb.set_trace()
        word_store.append(df)
    lms_combined = pd.concat(word_store, axis =1)
    lms_combined['sCounter'] = range(lms_combined.shape[0])
    lms_combined['sentence_index'] = i
    sentence_store.append(lms_combined)
wdf = pd.concat(sentence_store)

In [ ]:
wdf['word'] = wdf[u'bnc_unigram_word']

In [ ]:
wdf.head()

In [ ]:
# Each record is a 'produced' word (user_candidate_transcription) 

# Identifying Deleted and Changed Words

In [ ]:
# for generation n, get the indices of all words that have changed in n+1
# have changed: no longer appear? doesn't handle transpositions
# borrowed the function from the old version of telephone

In [ ]:
all_runs.loc[all_runs.user == "0"].head(5)

In [ ]:
# take a 2-column subset of all_trials that we can put into R
input_output = all_runs[['gold_candidate_transcription','user_candidate_transcription','user',
                         'upstream_subject_id']]
#gold_candidate_transcription is what a participant heard
#user_candidate_transcription is what the participant produced

# remove the intitial sentences -- these are represnted as input for the first participant
%R -i input_output

In [ ]:
input_output

In [ ]:
%%R
source('getWordLevenshteinDistance.R')
print(paste(nrow(input_output), 'sentences'))
names(input_output) = c('input','output','output_subject','input_subject')  
input_output$input = tolower(as.character(input_output$input))
input_output$output = tolower(as.character(input_output$output))

computeEditTable = function(s,r,input_subject, output_subject){
    if (s == 'none'){
        # this is an initial sentence, return NA
        return(NA)
    } else {
        et = getReducedEditTable(s,r)
        # for python compatibility, use 0-indices
        et$sCounter = et$sCounter - 1
        et$rCounter = et$rCounter - 1
        et$input_subject = input_subject
        et$output_subject = output_subject
        return(et)
    }
}

editTables = mapply(computeEditTable, input_output$input, input_output$output,
                   input_output$input_subject, input_output$output_subject)

In [ ]:
%%R
editTables[[2]] #this should correspond to wdf[,1]

In [ ]:
%%R
print(computeEditTable('this is a', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a', NULL, NULL ))
print(computeEditTable('this is a mouse', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a house', NULL, NULL))

In [ ]:
%%R
getDSMwrtInput = function(sentence_index, editTables){
    et = editTables[[sentence_index]]
    if (is.na(et)){
        return(data.frame(sentence_index, sCounter=NA,code=NA, input_subject=NA, output_subject=NA)) # these are initial sentences
    } else{
        et = editTables[[sentence_index]]    
        et$sentence_index = sentence_index
        # here is where insertions are removed so that we can join back with wfds
        return(subset(et, !is.na(sCounter))[,c('sentence_index','sCounter','code','input_subject','output_subject')])
    }
}

DSMwrtInput = do.call('rbind', lapply(c(1:length(editTables)), function(i){
    dsm = getDSMwrtInput(i, editTables)  
    if (!is.na(dsm)){
        dsm$sentence_index = dsm$sentence_index - 2 
        # -1 because Python indexes from 0
        # another -1 to bring the index of edits into alignment with the languag model results     
    } 
    return(dsm)
}))   

#DSMwrtInput = subset(DSMwrtInput, !is.na(code))

In [ ]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
DSMwrtInput = r['DSMwrtInput']
DSMwrtInput.iloc[0:15,]

In [ ]:
initialSentenceIndices = np.array(DSMwrtInput[np.isnan(DSMwrtInput.sCounter)]['sentence_index'].tolist())
wdf_initRemoved = wdf[(~wdf['sentence_index'].isin(initialSentenceIndices)) & (wdf['bnc_unigram_word'] != '</s>')]
DSMwrtInput_nansRemoved = DSMwrtInput[~np.isnan(DSMwrtInput.sCounter)]

In [ ]:
print(DSMwrtInput_nansRemoved.shape[0])
print(wdf_initRemoved.shape[0])
#assert(DSMwrtInput_nansRemoved.shape[0] == wdf_initRemoved.shape[0])

In [ ]:
wdf_initRemoved[wdf_initRemoved.sentence_index == 3192]

In [ ]:
DSMwrtInput_nansRemoved[DSMwrtInput_nansRemoved.sentence_index == 3192] 

In [ ]:
dsm_counts = DSMwrtInput_nansRemoved.groupby(['sentence_index']).sCounter.agg(np.size).reset_index()

dsm_counts.columns = ['sentence_index', 'dsm_count']
wdf_counts = wdf_initRemoved.groupby('sentence_index').sCounter.agg(np.size).reset_index()
wdf_counts.columns = ['sentence_index', 'wdf_count']
count_check = wdf_counts.merge(dsm_counts, how='outer')
count_check # differences in the counts?
count_check['difference'] = count_check.wdf_count - count_check.dsm_count
count_check.sort_values(by=['difference'], ascending=False)

In [ ]:
# merge DSMwrtInput into the word data frame
wdfr = wdf_initRemoved.merge(DSMwrtInput_nansRemoved) #sCounter is NaN for input sentences

In [ ]:
#where did all my happy data go?
wdfr.code.value_counts()

In [ ]:
wdfr

In [ ]:
wdfr.columns

# Merge With Word Properties

In [ ]:
wdfr['word'] = wdfr['bnc_unigram_word']
lexiconch = pd.read_csv('data/lexiconch.csv', index_col=0)
print('Number of words before merging with Lexiconch: '+str(wdfr.shape[0]))
wdfl = wdfr.merge(lexiconch, how='left')
print('Number of words after merging with Lexiconch: '+str(wdfl.shape[0]))
' '.join(set(wdfr.word) - set(wdfl.word))

In [ ]:
lexiconch.iloc[0]

In [ ]:
yarkoni_pld = pd.read_table('data/pld20.txt', header=None)
yarkoni_pld.columns = ['word', 'pld20']

In [ ]:
print('Number of words before merging with Yarkoni PLD: '+str(wdfl.shape[0]))
wdfy = wdfl.merge(yarkoni_pld, how='left')
print('Number of words after merging with Yarkoni PLD: '+str(wdfy.shape[0]))

In [ ]:
subtlex = pd.read_csv('data/subtlex_augmented.csv')
subtlex['word'] = subtlex.Word
subtlex.iloc[0]

In [ ]:
print('Number of words before merging with Subtlex: '+str(wdfy.shape[0]))
wdfx = wdfy.merge(subtlex[['word','SUBTLCD']], how='left')
print('Number of words after merging with Subtlex: '+str(wdfx.shape[0]))

# Merge in Sentence Level Predictors

In [ ]:
all_runs['sentence_index'] = range(all_runs.shape[0])

In [ ]:
#wdfs = wdfr.merge(all_runs, on='sentence_index') # !!! temporarily remove additional predictors
wdfs = wdfx.merge(all_runs, on='sentence_index') 

In [ ]:
wdfs.loc[wdfs.sentence_index == 1][['bnc_unigram_word',
                                   'user_candidate_transcription',
                                    'gold_candidate_transcription',
                                    'input_subject',
                                    'output_subject'
                                   ]]

In [ ]:
' '.join(wdfs.columns)

# Save the Output

In [ ]:
wdfs.shape

In [ ]:
wdfs.to_csv('output/wordLevelChanges.csv', index=False)

See R notebook, Error Prediction Model - Logistic Regression